In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
flow = pd.read_csv("10_nodes_flow.csv",header = None)
cost = pd.read_csv("10_nodes_cost.csv",header = None)

In [3]:
flow

,0,1,2,3,4,5,6,7,8,9
0,0,6469,7629,20036,4690,6194,11688,2243,8857,7248
1,6469,0,12999,13692,3322,5576,3878,3202,6699,4198
2,7629,12999,0,35135,5956,14121,5951,5768,16578,4242
3,20036,13692,35135,0,19094,35119,21423,27342,51341,15826
4,4690,3322,5956,19094,0,7284,3102,1562,7180,1917
5,6194,5576,14121,35119,7284,0,5023,3512,10419,3543
6,11688,3878,5951,21423,3102,5023,0,11557,6479,34261
7,2243,3202,5768,27342,1562,3512,11557,0,5615,7095
8,8857,6699,16578,51341,7180,10419,6479,5615,0,4448
9,7248,4198,4242,15826,1917,3543,34261,7095,4448,0


In [4]:
total_flow = (flow.sum()).sum()/2
total_flow

499513.0

In [5]:
cost

,0,1,2,3,4,5,6,7,8,9
0,0.0000,576.9631,946.4954,597.5972,373.8127,559.7673,709.0215,1208.3280,603.6477,695.2080
1,576.9631,0.0000,369.5327,613.0386,429.1079,312.8831,1196.4890,1502.1400,405.8975,1241.9610
2,946.4954,369.5327,0.0000,858.3308,749.6018,556.0706,1541.2730,1764.7910,621.3306,1603.1650
3,597.5972,613.0386,858.3308,0.0000,255.0303,311.3071,790.1213,907.4331,237.0703,932.2173
4,373.8127,429.1079,749.6018,255.0303,0.0000,225.8954,794.1726,1080.3740,238.9440,879.5647
5,559.7673,312.8831,556.0706,311.3071,225.8954,0.0000,1009.6890,1216.8680,94.2588,1104.5740
6,709.0215,1196.4890,1541.2730,790.1213,794.1726,1009.6890,0.0000,663.8762,982.7378,221.4220
7,1208.3280,1502.1400,1764.7910,907.4331,1080.3740,1216.8680,663.8762,0.0000,1143.7910,874.5181
8,603.6477,405.8975,621.3306,237.0703,238.9440,94.2588,982.7378,1143.7910,0.0000,1094.9060
9,695.2080,1241.9610,1603.1650,932.2173,879.5647,1104.5740,221.4220,874.5181,1094.9060,0.0000


In [6]:
random.randint(0,9)

7

In [41]:
def original_solution(n):
    hub = []
    while len(hub) < n:
        a = random.randint(0,9)
        if a not in hub:
            hub.append(a)
    solution = np.arange(10)
    for i in range(0,10):
        b = random.randint(0,2)
        if i in hub:
            solution[i] = i
        else:
            solution[i] = hub[b]
    return hub,solution

original_solution = original_solution(3)
hub = original_solution[0]
solution = original_solution[1]
print(hub)
print(solution)

[6, 1, 4]
[4 1 4 4 4 1 6 4 1 6]


In [9]:
def find_total_cost(solution,alpha):
    dis = alpha
    total_cost = 0
    for m in range(len(cost)):
        for n in range(0,m):
            a = solution[m]
            b = solution[n]
            if a == b:
                solution_cost = flow[m][n] * (cost[m][a] + cost[b][n])
            else:
                solution_cost = flow[m][n] * (cost[m][a] + cost[a][b] * dis + cost[b][n])
            total_cost = total_cost + solution_cost
    return total_cost

In [10]:
find_total_cost(solution,0.2)

511183780.5858801

# tabu search ##

In [37]:
#tabu search
def ts(hub,solution,alpha,max_tik):
    
    def candidate(hub,solution,tabu_list):
        hub1 = hub
        candidates = []
       # print(solution)
        for i in range(len(solution)):
            origin_hub = hub1.copy()
            base_solution = solution.copy()
            if i != base_solution[i]:
                origin_hub.remove(base_solution[i])
                for j in origin_hub:
                    if i not in tabu_list:
                        base_solution[i] = j
                        waiting_list = base_solution.copy()
                        candidates.append(waiting_list)
        return candidates
    
    def find_best_candidate(candidates):
        potential_candidate = candidates.copy()
        total_cost = find_total_cost(potential_candidate[0],alpha)
        best_candidate = potential_candidate[0]
        for i in potential_candidate:
            candidate_cost = find_total_cost(i,alpha)
            if candidate_cost < total_cost:
                best_candidate = i
                total_cost = candidate_cost
        print(best_candidate)
        return best_candidate
    
    def check(tabu_list,solution_old,solution_new,list_length,tik):
    # check if the solution is in the tabu list
        lowest_cost = find_total_cost(solution_old,alpha)
        total_cost = find_total_cost(solution_new,alpha)
        for i in range(len(solution_old)):
            if solution_new[i] != solution_old[i]:
                change = i
        if change not in tabu_list:
            solution = solution_new
            tabu_list.append(change)
            tik = tik + 1
            if len(tabu_list) > list_length:
                del tabu_list[0]
        elif tik == max_tik and change in tabu_list and total_cost < lowest_cost:
            solution = solution_new
            tik = tik + 1
            tabu_list.remove(change)
        else:
            solution = solution_old
            
        return solution,tabu_list,tik  
        
    tik = 0
    tabu_list = []
    tabu_length = 5
    overall_best_solution = solution.copy()
    while tik < max_tik:  
        candidates = candidate(hub,solution,tabu_list)
        best_candidate = find_best_candidate(candidates)
        if find_total_cost(best_candidate,alpha) < find_total_cost(overall_best_solution,alpha):
            overall_best_solution = best_candidate
        iteration = check(tabu_list,solution,best_candidate,tabu_length,tik)
        tabu_list = iteration[1]
        solution = iteration[0]
        tik = iteration[2]
    
    final_solution = overall_best_solution
    final_cost = find_total_cost(overall_best_solution,alpha)    
    return final_solution,final_cost
              
final_tabu_cost = ts([3,5,6], [6,3,3,3,3,5,6,5,5,5] , 0.2, 10)
final_tabu_cost 

[6, 3, 3, 3, 3, 5, 6, 5, 5, 6]
[6, 3, 3, 3, 3, 5, 6, 6, 5, 6]
[6, 3, 5, 3, 3, 5, 6, 6, 5, 6]
[6, 5, 5, 3, 3, 5, 6, 6, 5, 6]
[5, 5, 5, 3, 3, 5, 6, 6, 5, 6]
[5, 5, 5, 3, 5, 5, 6, 6, 5, 6]
[5, 5, 5, 3, 5, 5, 6, 6, 3, 6]
[5, 5, 5, 3, 5, 5, 6, 3, 3, 6]
[5, 5, 3, 3, 5, 5, 6, 3, 3, 6]
[5, 3, 3, 3, 5, 5, 6, 3, 3, 6]


([5, 5, 5, 3, 5, 5, 6, 6, 5, 6], 245727593.58790004)

In [38]:
optimal_tabu = final_tabu_cost[1]/total_flow
optimal_tabu

491.9343312144029

# simulated annealing

In [39]:
#simulated annealing

def sa(hub,solution,alpha,initial_theta,max_tik,lamda,r_initial):

    def candidate(hub,solution):
        hub1 = hub
        candidates = []
        for i in range(len(solution)):
            origin_hub = hub1.copy()
            base_solution = solution.copy()
            if i != base_solution[i]:
                origin_hub.remove(base_solution[i])
                for j in origin_hub:    
                    base_solution[i] = j
                    waiting_list = base_solution.copy()
                    candidates.append(waiting_list)
        return candidates
    
    tik = 1
    theta = initial_theta
    initial_solution = solution.copy()
    solution_i = initial_solution
    best_solution = solution_i
    r = r_initial
    while tik <= max_tik:
        candidates = candidate(hub,solution_i)
        for i in candidates:
            if find_total_cost(i,alpha) < find_total_cost(best_solution,alpha):
                best_solution = i
            if find_total_cost(i,alpha) < find_total_cost(solution_i,alpha):
                solution_i = i 
            else:
                ran = random.random()
                delta = find_total_cost(solution_i,alpha) - find_total_cost(i,alpha)
                p = math.exp(delta/theta)
                if ran < p:
                    solution_i = i
                if delta > 0:
                    r = r + 1
                elif delta < 0:
                    r = 0
            theta = theta + lamda * math.log(1 + r)
            tik = tik + 1
    
    return find_total_cost(best_solution,alpha)
                
final_sa_cost = sa([3,5,6], [6,3,3,3,3,5,6,5,5,5],0.2,800000,100,0,0.8)                
final_sa_cost


245727593.58790004

In [40]:
optimal_sa = final_sa_cost/total_flow
optimal_sa

491.9343312144029

# variable neighbourhood search

In [15]:
def neighbourhood_structure(solution,hub,n):
        
    if n == 1:
        # change initial hub
        candidates = []
        for i in range(len(solution)):
            candidate = solution.copy()
            initial_hub = candidate[i]
            if i != candidate[i]:
                for j in range(len(solution)):
                    if candidate[j] == initial_hub:
                        candidate[j] = i
                waiting_list = candidate.copy()
                candidates.append(waiting_list)
        return candidates
          
            
    elif n == 2:
        #swap two nodes' hubs
        candidates = []
        for i in range(len(solution)):
            if i != solution[i]:
                for j in range(i,len(solution)):
                    solution_k = solution.copy()
                    if j != solution[j] and solution[i] != solution[j]:
                        subtitute = solution_k[i]
                        solution_k[i] = solution_k[j]
                        solution_k[j] = subtitute
                        waiting_list = solution_k.copy()
                        candidates.append(waiting_list)
        return candidates

    
    elif n == 3:
        #change hub for one node
        candidates = []
        for i in range(len(solution)):
            origin_hub = hub.copy()
            base_solution = solution.copy()
            if i != base_solution[i]:
                origin_hub.remove(base_solution[i])
                for j in origin_hub:    
                    base_solution[i] = j
                    waiting_list = base_solution.copy()
                    candidates.append(waiting_list)
    
        return candidates
        

    elif n == 4:
        #swap two hubs's nodes
        candidates = []
        candidate = solution.copy()
        base_hub = hub.copy()
        for i in range(len(hub)):
            if i != len(base_hub)-1:
                subtitute = base_hub[i+1]
            else:
                subtitute = base_hub[0]
            for j in range(len(solution)):
                if candidate[j] == subtitute and j != candidate[j]:
                    candidate[j] = base_hub[i]
                    if j != candidate[j] and candidate[j] == base_hub[i]:
                        candidate[j] = subtitute
            waiting_list = candidate.copy()
            candidates.append(waiting_list)
        return candidates
            
        
def find_best_candidate(candidates,alpha):
    potential_candidate = candidates.copy()
    total_cost = find_total_cost(potential_candidate[0],alpha)
    best_candidate = potential_candidate[0]
    for i in potential_candidate:
        candidate_cost = find_total_cost(i,alpha)
        if candidate_cost < total_cost:
            best_candidate = i
            total_cost = candidate_cost
    return best_candidate


In [42]:
#variable neighbourhood descent            
def vnd(hub,solution,max_tik,alpha):    
    solution_i = solution.copy()
    best_solution = solution_i
    tik = 1
    while tik <= max_tik:
        for i in range(1,5):
            candidates = neighbourhood_structure(solution_i,hub,i)
            #print(candidates)
            best_candidate = find_best_candidate(candidates,alpha)
            #print(best_candidate)
            #print(list(set(best_candidate)))
            if find_total_cost(best_candidate,alpha) < find_total_cost(solution_i,alpha):
                solution_i = best_candidate
            elif find_total_cost(best_candidate,alpha) < find_total_cost(best_solution,alpha):
                best_solution = best_candidate    
            hub = list(set(solution_i))
        tik = tik + 1
    return best_solution

vnd_solution = vnd(hub,solution,50,0.2)
final_vnd_cost = find_total_cost(vnd_solution,0.2)
print(vnd_solution)
vnd_hub = list(set(vnd_solution))
print(vnd_hub)
optimal_vnd = final_vnd_cost/total_flow
optimal_vnd

[5 5 5 3 5 5 6 6 5 6]
[3, 5, 6]


491.9343312144029

In [56]:
#RVNS(结果不变但结果错误)
def rvns(hub,solution,max_tik,alpha):    
    solution_i = solution.copy()
    best_solution = solution_i
    tik = 1
    k = 1
    while tik <= max_tik:
        candidates = neighbourhood_structure(solution_i,hub,k)
        #print(candidates)
        ran = random.randint(0,len(candidates)-1)
        candidate = candidates[ran]
        print(candidate)
        #print(k)
        if find_total_cost(candidate,alpha) < find_total_cost(candidate,alpha):
            solution_i = candidate
            k = 1
            if find_total_cost(candidate,alpha) < find_total_cost(best_solution,alpha):
                best_solution = candidate
        else:
            if k == 4:
                k = 1
            else:  
                k = k + 1
        hub = list(set(solution_i))
        tik = tik + 1
    return best_solution

rvns_solution = rvns(hub,solution,50,0.2)
final_rvns_cost = find_total_cost(rvns_solution,0.2)
print(rvns_solution)
rvns_hub = list(set(rvns_solution))
print(rvns_hub)
optimal_rvns = final_rvns_cost/total_flow
optimal_rvns

[0 1 0 0 0 1 6 0 1 6]
[4 1 4 1 4 4 6 4 1 6]
[4 1 4 4 4 1 6 6 1 6]
[4 1 4 4 4 1 6 4 1 6]
[2 1 2 2 2 1 6 2 1 6]
[4 1 4 4 4 1 6 4 6 1]
[4 1 6 4 4 1 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[4 1 4 4 4 1 9 4 1 9]
[4 1 4 1 4 1 6 4 4 6]
[6 1 4 4 4 1 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[2 1 2 2 2 1 6 2 1 6]
[4 1 1 4 4 4 6 4 1 6]
[4 1 4 4 4 1 6 6 1 6]
[4 1 4 4 4 1 6 4 1 6]
[4 8 4 4 4 8 6 4 8 6]
[1 1 4 4 4 1 6 4 4 6]
[4 1 4 4 4 1 6 6 1 6]
[4 1 4 4 4 1 6 4 1 6]
[4 8 4 4 4 8 6 4 8 6]
[4 1 4 4 4 1 6 1 4 6]
[4 1 4 4 4 4 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[3 1 3 3 3 1 6 3 1 6]
[4 1 4 4 4 1 6 6 1 4]
[1 1 4 4 4 1 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[2 1 2 2 2 1 6 2 1 6]
[4 1 4 6 4 1 6 4 1 4]
[4 1 1 4 4 1 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[3 1 3 3 3 1 6 3 1 6]
[4 1 4 4 4 1 6 4 6 1]
[4 1 4 4 4 1 6 6 1 6]
[4 1 4 4 4 1 6 4 1 6]
[4 8 4 4 4 8 6 4 8 6]
[4 1 6 4 4 1 6 4 1 4]
[6 1 4 4 4 1 6 4 1 6]
[4 1 4 4 4 1 6 4 1 6]
[7 1 7 7 7 1 6 7 1 6]
[4 1 4 6 4 1 6 4 1 4]
[4 1 4 4 4 1 6 6 1 6]
[4 1 4 4 4 1 6 4 1 6]
[0 1 0 0 0 1 6 0 1 6]
[6 1 4 4 4

752.2474678906856

In [53]:
#SVNS
def svns(solution,hub,max_tik,alpha,gamma):
    solution_i = solution.copy()
    best_solution = solution.copy()
    hub_i = hub.copy()
    tik = 0
    while tik <= max_tik:
        for i in range(1,5):
            candidates = neighbourhood_structure(solution_i,hub_i,i)
            ran = random.randint(0,len(candidates)-1)
            candidate = candidates[ran]
            #print(candidate)
            if find_total_cost(candidate,alpha) < find_total_cost(solution_i,alpha):
                solution_i = candidate
            candidates_2 = neighbourhood_structure(candidate,hub_i,i)
            best_candidate = find_best_candidate(candidates_2,alpha)
            if find_total_cost(best_candidate,alpha) < find_total_cost(solution_i,alpha):
                solution_i = best_candidate
                if find_total_cost(best_candidate,alpha) < find_total_cost(best_solution,alpha):
                    best_solution = best_candidate
            elif find_total_cost(best_candidate,alpha) - gamma * (find_total_cost(best_candidate,alpha) - find_total_cost(solution_i,alpha)) < find_total_cost(solution_i,alpha):
                    solution_i = best_candidate
            hub_i = list(set(solution_i))
            tik = tik + 1
                    
    return best_solution
            
svns_solution = svns(solution,hub,50,0.2,0.2)
final_svns_cost = find_total_cost(svns_solution,0.2)
print(svns_solution)
svns_hub = list(set(svns_solution))
print(svns_hub)
optimal_svns = final_svns_cost/total_flow
optimal_svns        


[5 5 5 3 5 5 6 6 5 6]
[3, 5, 6]


491.9343312144029

In [45]:
#RRVNS
def rrvns(solution,hub,max_tik,alpha):
    tik = 0
    solution_i = solution.copy()
    hub_i = hub.copy()
    best_solution = solution_i
    while tik <= max_tik:
        for i in range(1,5):
            candidates = neighbourhood_structure(solution_i,hub_i,i)
            #print(candidates)
            best_candidate = find_best_candidate(candidates,alpha)
            if find_total_cost(best_candidate,alpha) < find_total_cost(solution_i,alpha):
                solution_i = best_candidate
                hub_i = list(set(solution_i))
        if find_total_cost(solution_i,alpha) < find_total_cost(best_solution,alpha):
            best_solution = solution_i
        tik = tik + 1
    return best_solution
    
rrvns_solution = rrvns(solution,hub,50,0.2)
final_rrvns_cost = find_total_cost(rrvns_solution,0.2)
print(rrvns_solution)
rrvns_hub = list(set(rrvns_solution))
print(rrvns_hub)
optimal_rrvns = final_rrvns_cost/total_flow
optimal_rrvns

[5 5 5 3 5 5 6 6 5 6]
[3, 5, 6]


491.9343312144029